In [66]:
print("ok")

ok


In [67]:
%pwd

'/Users/chaitanyagarg/Desktop/VS Code /Medical-Chatbot-'

In [68]:
import os 
os.chdir("../")

In [69]:
%pwd

'/Users/chaitanyagarg/Desktop/VS Code '

In [70]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [71]:
# Extra Data from PDF File 
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [72]:
extracted_data = load_pdf_file(data='Data/')


In [73]:
# extracted_data

In [74]:
#Split the Data into Text Chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks 

In [75]:
text_chunks=text_split(extracted_data)
print("Legnth of Text Chunks :- ",len(text_chunks))

Legnth of Text Chunks :-  0


In [76]:
text_chunks

[]

In [77]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings


In [78]:
embeddings = download_hugging_face_embeddings()

In [79]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [80]:
# query_result

In [81]:
from dotenv import load_dotenv
load_dotenv()

False

In [82]:
PANECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [88]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os 

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

Exception: The official Pinecone python package has been renamed from `pinecone-client` to `pinecone`. Please remove `pinecone-client` from your project dependencies and add `pinecone` instead. See the README at https://github.com/pinecone-io/pinecone-python-client for more information on using the python SDK.